In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')    # reading the csv file

train.head()      # printing first five rows of the file

In [ ]:
train['Pawpularity'] = train['Pawpularity'] /100.
train.head()

In [ ]:
train.columns

In [ ]:
img_size = 140
train_image = []
for i in tqdm(range(train.shape[0])):#train.shape[0]
    img = image.load_img('../input/petfinder-pawpularity-score/train/'+train['Id'][i]+'.jpg',
                         target_size=(img_size,img_size,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

In [ ]:
X.shape

In [ ]:
plt.imshow(X[5])

In [ ]:
train['Pawpularity'][2]

In [ ]:
y = np.array(train.drop(['Id'],axis=1))#, 'Pawpularity'
# y=y[0:2000]
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
y_train[0:4]

In [ ]:
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
inputs = keras.Input(shape=(img_size, img_size, 3))

X = Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu')(inputs)
X = MaxPool2D(pool_size=(2, 2))(X)

X = Conv2D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu')(X)
X = MaxPool2D(pool_size=(2, 2))(X)

X = Conv2D(filters=256, kernel_size=3, strides=2, padding='same', activation='relu')(X)
# X = MaxPool2D(pool_size=(2, 2))(X)

#X = Flatten()(X)

X = GlobalAveragePooling2D()(X)

X = Dense(512, activation='relu')(X)
X = Dropout(0.3)(X)

outputs = Dense(13)(X)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:

# model = Sequential()
# model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(128,128,3)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# # model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))
# model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# # model.add(Dropout(0.25))

# # model.add(Flatten())
# model.add(GlobalAveragePooling2D())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(13))#, activation='sigmoid'

In [ ]:
model.summary()

In [ ]:
LEARNING_RATE = 0.001
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),#'adam',
              loss='msle',#'mse',
              metrics=['accuracy',RootMeanSquaredError()])#binary_crossentropy


In [ ]:
model.fit(X_train, y_train, epochs=64, validation_data=(X_test, y_test), batch_size=64)

In [ ]:
from pathlib import Path
path = Path('../input/petfinder-pawpularity-score')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
df_test = pd.read_csv(path / 'test.csv', index_col='Id')

test_path = path / 'test'

df_test.loc[:, 'filename'] = df_test.index + '.jpg'
RESCALE = 1./255

datagen = ImageDataGenerator(rescale=RESCALE)

test_generator = datagen.flow_from_dataframe(dataframe=df_test,
                                             directory=test_path, 
                                             x_col='filename',
                                             y_col=None,
                                             batch_size=32,
                                             seed=42,
                                             shuffle=False,
                                             class_mode=None,
                                             target_size=(img_size, img_size)
                                            )


In [ ]:
yhat_test = model.predict(test_generator, verbose=1)

In [ ]:
yhat_test

In [ ]:
yhat_test_1 = yhat_test[:,-1]
print(yhat_test_1)

In [ ]:
frames = [
    pd.Series(df_test.index.values, name='Id'),
    pd.Series(yhat_test_1.ravel(), name='Pawpularity')
]
submission = pd.concat(frames, axis=1)
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)